In [42]:
from os import name

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [43]:
df = pd.read_csv('train.txt', sep=';', header=None,names = ['text','emotion'])

In [44]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [45]:
df.isnull().sum()

text       0
emotion    0
dtype: int64

In [46]:
unique_emotions = df['emotion'].unique()
emotion_numbers = {}
i = 0
for emo in unique_emotions:
    emotion_numbers[emo] = i
    i +=1
df['emotion'] = df['emotion'].map(emotion_numbers)

In [47]:
df['text'] = df['text'].apply(lambda x : x.lower())

In [48]:
df.head()

,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1


In [49]:
import string

In [50]:
def remove_punc(text):
    return text.translate(str.maketrans('', '',string.punctuation))

In [51]:
df['text'] = df['text'].apply(remove_punc)

In [52]:

def remove_numbers(txt):
    new = ""
    for i in txt:
        if not i.isdigit():
            new = new + i
    return new

df['text'] = df['text'].apply(remove_numbers)

In [53]:

def remove_emojis(txt):
    new = ""
    for i in txt:
        if i.isascii():
            new += i
    return new

df['text'] = df['text'].apply(remove_emojis)

In [54]:
import nltk

In [55]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [56]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
stop_words = set(stopwords.words('english'))

In [58]:
def remove(text):
    words = text.split()
    cleaned = []
    for i in words:
        if not i in stop_words:
            cleaned.append(i)
    return ' '.join(cleaned)


In [59]:
df['text'] = df['text'].apply(remove)


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['emotion'], test_size=0.20, random_state=42)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [62]:
bow_vectorizer = CountVectorizer()

In [63]:
X_train_bow = bow_vectorizer.fit_transform(X_train)

In [64]:
X_train_bow

<12800x13361 sparse matrix of type '<class 'numpy.int64'>'
	with 116059 stored elements in Compressed Sparse Row format>

In [65]:
X_test_bow = bow_vectorizer.transform(X_test)

In [66]:
nb_model = MultinomialNB()

In [67]:
nb_model.fit(X_train_bow,y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [69]:
pred_bow = nb_model.predict(X_test_bow)
print(accuracy_score(y_test, pred_bow))

0.768125


In [70]:


tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


nb2_model = MultinomialNB()
nb2_model.fit(X_train_tfidf,y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [71]:
y_pred = nb2_model.predict(X_test_tfidf)

In [72]:

print(accuracy_score(y_test, y_pred))

0.6609375


In [73]:
from sklearn.linear_model import LogisticRegression

In [78]:
lr = LogisticRegression(max_iter=100000)

In [79]:
lr.fit(X_train_tfidf,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100000
,multi_class,'deprecated'


In [80]:

log_pred = lr.predict(X_test_tfidf)

In [81]:

print(accuracy_score(y_test,log_pred ))

0.8628125
